In [1]:
import numpy as np

In [2]:
# Output=32, Input=64
input_size = 64
output_size = 32
W = np.random.rand(input_size, output_size)
b = np.random.rand(output_size)
X = np.random.rand(input_size)
print(f'shape of X: {X.shape}, W: {W.shape}, B: {b.shape}')
Y = X @ W + b
print(f'output shape: {Y.shape}')

shape of X: (64,), W: (64, 32), B: (32,)
output shape: (32,)


In [3]:
# forward
# batch dimention included
B = 16
X = np.random.rand(B, input_size)
print(f'shape of X: {X.shape}, W: {W.shape}, B: {b.shape}')
Y = X @ W + b
print(f'output shape: {Y.shape}')
print(Y.shape)

shape of X: (16, 64), W: (64, 32), B: (32,)
output shape: (16, 32)
(16, 32)


In [4]:
dLdout = np.random.rand(output_size)
X = np.random.rand(input_size)
dLdW_1 = np.outer(X, dLdout)

# (I, 1) dot (1, O) -> (I, O)
dLdW_2 = X[:, None] @ dLdout[None, :]
print(f'shape of X[:, None]: {X[:, None].shape}')
print(f'shape of dout[None, :]: {dLdout[None, :].shape}')
print(dLdW_1 - dLdW_2)

shape of X[:, None]: (64, 1)
shape of dout[None, :]: (1, 32)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# backward
# X (N, I) dLdout (N, O)
# X (I) dLdout (O)
# dLdw[i][o] = X[i] * dLdout[o]

X = np.random.rand(B, input_size)
# dLdout = np.random.rand(B, output_size)
dLdout = np.ones((B, output_size))


# (1, I)' dot (1, O) -> (I, O)
X[0:1,].transpose() @ dLdout[0:1,]

dLdw = X.transpose() @ dLdout
dLdw_einsum = np.einsum('ni,no->io', X, dLdout)

print(f'shape of dLdw {dLdw.shape}')
print(f'shape of dLdw_einsum {dLdw_einsum.shape}')

print((dLdw - dLdw_einsum))
# print(dLdw)
# print(dLdw_einsum)

shape of dLdw (64, 32)
shape of dLdw_einsum (64, 32)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
# trace of A
A = np.random.rand(5, 5)
print(np.einsum('ii', A))
print(np.trace(A))

2.374747556357096
2.374747556357096


In [8]:
# print(np.diag(A)) 
# print(np.einsum('ii->i', A))
# extract diagnonal element
A = np.random.rand(5, 5, 5)
print(np.einsum('nii->ni', A)[0])
print(np.diag(A[0]))

[0.67850179 0.71502252 0.55310526 0.69844084 0.32049398]
[0.67850179 0.71502252 0.55310526 0.69844084 0.32049398]


In [9]:
np.diag([1, 2, 3, 4])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

In [10]:
A = np.arange(10).reshape(2, 5)
A_t = np.einsum('ij->ji', A)
print(A_t)
print(A.transpose())

[[0 5]
 [1 6]
 [2 7]
 [3 8]
 [4 9]]
[[0 5]
 [1 6]
 [2 7]
 [3 8]
 [4 9]]


In [51]:
# softmax
X = np.random.rand(5)
dLdS = np.random.rand(5)

def sample_softmax_backward(X, dLdS):
    X_exp = np.exp(X)
    S = X_exp / np.sum(X_exp)
    diag_S = np.einsum('o,os->os', S, np.eye(S.shape[0]))

    SS_T = np.outer(S, S)
    SS_T = np.einsum('no, ni->nio', S[None, :], S[None, :])
    dSdX = diag_S - SS_T
    # downstream gradient
    # dSdX (O, O) dLdS (O) dLdX (O)
    dLdX = dSdX @ dLdS

    return dLdX

In [52]:
# softmax with batch
N = 4
X = np.random.rand(N, 5)
dLdS = np.random.rand(N, 5)
def batch_softmax_backward(X, dLdS):
    X_exp = np.exp(X)
    # keepdims for broadcasting
    S = X_exp / np.sum(X_exp, axis=1, keepdims=True)
    diag_S = np.einsum('no,oi->nio', S, np.eye(S.shape[1]))

    # SS_T = np.outer(S, S)
    SS_T = np.einsum('no, ni->nio', S, S)

    dSdX = diag_S - SS_T
    # downstream gradient
    # dSdX (N, I, O) dLdS (N, O) dLdX (N, I)
    # dLdX (N, I, N)
    # dLdX = dSdX @ dLdS.transpose()
    dLdX = np.einsum('nio,no->ni', dSdX, dLdS)
    print(dLdX.shape)

    return dLdX


In [67]:
N = 4
X = np.random.rand(N, 5)
dLdS = np.random.rand(N, 5)

dLdW_1 = sample_softmax_backward(X[0], dLdS[0])
dLdW_2 = batch_softmax_backward(X, dLdS)[0]
print(dLdW_1)
print(dLdW_2)

(4, 5)
[ 0.00268701 -0.02608164 -0.02266046  0.04690991 -0.00085482]
[ 0.00268701 -0.02608164 -0.02266046  0.04690991 -0.00085482]


In [4]:
input_channel = 64
output_channel = 8
batch_size = 64
w = 3; h = 3
conv_kernel = np.random.rand(output_channel, input_channel, w, h)
patch = np.random.rand(batch_size, input_channel, w, h)

In [5]:
def conv2d_slow():
    conv = np.random.rand(8, 64, 3, 3)
    patch = np.random.rand(64, 64, 3, 3)
    results = np.zeros((64, 8, 1, 1))
    for ii, c in enumerate(conv):
        for jj, p in enumerate(patch):
            results[jj, ii] = np.sum(c * p)

In [6]:
def conv2d_fast():
    conv = np.random.rand(8, 64, 3, 3)
    patch = np.random.rand(64, 64, 3, 3)
    conv_flat = conv.reshape(8, -1)
    patch_flat = patch.reshape(64, -1)
    result = conv_flat @ patch_flat.transpose()

In [7]:
def conv2d_einsum():
    conv = np.random.rand(8, 64, 3, 3)
    patch = np.random.rand(64, 64, 3, 3)
    result = np.einsum('oiwh,biwh->bo', conv, patch)

In [8]:
%timeit -n 10 conv2d_fast()

430 µs ± 134 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit -n 10 conv2d_einsum()

351 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit -n 10 conv2d_slow()

3.21 ms ± 52.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
conv = np.random.rand(3, 4, 3, 3)
patch = np.random.rand(8, 4, 3, 3)
np.einsum('oiwh,biwh->bo', conv, patch)

array([[ 9.57228531,  8.80686831, 10.09557386],
       [11.44700331,  9.48999174, 11.31632708],
       [ 9.36179557,  9.40877549,  9.01509729],
       [ 9.74014882,  8.44165404, 10.38283865],
       [10.09371035,  9.71329664, 10.84139362],
       [ 9.60616568,  8.4117137 ,  9.87156126],
       [ 8.65563611,  8.15354132,  9.03243714],
       [ 7.92858488,  7.16413446,  8.15754467]])

In [12]:
results = np.zeros((8, 3))
for ii, c in enumerate(conv):
    for jj, p in enumerate(patch):
        results[jj, ii] = np.sum(c * p)
results

array([[ 9.57228531,  8.80686831, 10.09557386],
       [11.44700331,  9.48999174, 11.31632708],
       [ 9.36179557,  9.40877549,  9.01509729],
       [ 9.74014882,  8.44165404, 10.38283865],
       [10.09371035,  9.71329664, 10.84139362],
       [ 9.60616568,  8.4117137 ,  9.87156126],
       [ 8.65563611,  8.15354132,  9.03243714],
       [ 7.92858488,  7.16413446,  8.15754467]])

In [13]:
conv_flat = conv.reshape(3, -1)
patch_flat = patch.reshape(8, -1)
results = patch_flat @ conv_flat.transpose()
results

array([[ 9.57228531,  8.80686831, 10.09557386],
       [11.44700331,  9.48999174, 11.31632708],
       [ 9.36179557,  9.40877549,  9.01509729],
       [ 9.74014882,  8.44165404, 10.38283865],
       [10.09371035,  9.71329664, 10.84139362],
       [ 9.60616568,  8.4117137 ,  9.87156126],
       [ 8.65563611,  8.15354132,  9.03243714],
       [ 7.92858488,  7.16413446,  8.15754467]])

In [65]:
import numpy as np

def im2col(X, kernel_size, stride, padding):
    batch_size, in_channels, in_height, in_width = X.shape
    
    # Add padding to the input tensor
    padded_X = np.pad(X, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
    
    out_height = (in_height + 2 * padding - kernel_size) // stride + 1
    out_width = (in_width + 2 * padding - kernel_size) // stride + 1
    
    # Reshape the input tensor to a matrix
    X_col = np.zeros((in_channels * kernel_size * kernel_size, batch_size, out_height * out_width))
    for i in range(out_height):
        for j in range(out_width):
            patch = padded_X[:, :, i*stride:i*stride+kernel_size, j*stride:j*stride+kernel_size]
            patch_col = patch.reshape(batch_size, -1).transpose(1, 0)
            X_col[:, :, (i*out_width+j)] = patch_col
                
    return X_col.reshape(in_channels*kernel_size*kernel_size, batch_size*out_height*out_width)

class ConvLayerIm2col:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize the weights and biases
        self.weights = np.ones((out_channels, in_channels, kernel_size, kernel_size))
        self.biases = np.zeros((out_channels, 1))
        
    def forward(self, X):
        # Save the input tensor for the backward pass
        self.X = X
        
        # Compute the output tensor using
        # the im2col function
        X_col = im2col(X, self.kernel_size, self.stride, self.padding)
        
        # Flatten the weights tensor to perform a matrix multiplication
        W_col = self.weights.reshape(self.out_channels, -1)
        
        # Compute the matrix multiplication and add the biases
        out = np.dot(W_col, X_col) + self.biases
        
        # Reshape the output tensor to the correct shape
        batch_size = X.shape[0]
        out_height = (X.shape[2] + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (X.shape[3] + 2 * self.padding - self.kernel_size) // self.stride + 1
        out = out.reshape(self.out_channels, batch_size, out_height, out_width)
        out = np.transpose(out, (1, 0, 2, 3))
        
        return out

class ConvLayerFor:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize the weights and biases
        self.weights = np.ones((out_channels, in_channels, kernel_size, kernel_size))
        self.biases = np.zeros((out_channels, 1))
        
    def forward(self, X):
        # Save the input tensor for the backward pass
        padding = self.padding
        stride = self.stride
        kernel_size = self.kernel_size
        out_channels = self.out_channels
        batch_size, in_channel, in_height, in_width = X.shape
        padded_X = np.pad(X, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
        
        out_height = (in_height + 2 * padding - kernel_size) // stride + 1
        out_width = (in_width + 2 * padding - kernel_size) // stride + 1
        
        # Compute the output tensor using
        # the im2col function
        out = np.zeros((batch_size, out_channels, out_height, out_width))
        for i in range(out_height):
            for j in range(out_width):
                patch = padded_X[:, :, i*stride:i*stride+kernel_size, j*stride:j*stride+kernel_size]
                out[:, :, i, j] = np.einsum('oihw,bihw->bo', 
                                            self.weights, 
                                            patch)
        
        # Reshape the output tensor to the correct shape
        
        return out


In [66]:
conv1 = ConvLayerIm2col(4, 10, 3, padding=1)
conv2 = ConvLayerFor(4, 10, 3, padding=1)
X = np.ones((4, 4, 5, 5))
out1 = conv1.forward(X)
out2 = conv2.forward(X)
# conv.backward(np.ones((4, 10, 5, 5)))

In [73]:
out1[0, 0]

array([[16., 24., 24., 24., 16.],
       [24., 36., 36., 36., 24.],
       [24., 36., 36., 36., 24.],
       [24., 36., 36., 36., 24.],
       [16., 24., 24., 24., 16.]])

In [74]:
out2[0, 0]

array([[16., 24., 24., 24., 16.],
       [24., 36., 36., 36., 24.],
       [24., 36., 36., 36., 24.],
       [24., 36., 36., 36., 24.],
       [16., 24., 24., 24., 16.]])

In [71]:
%timeit -n 10 conv1.forward(X)

128 µs ± 7.88 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%timeit -n 10 conv2.forward(X)

320 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
